# Frequency-Domain Master Curve Validation

Validate RheoJAX oscillatory fits using the PyVisco master curve dataset.

In [1]:
# Google Colab compatibility - uncomment if running in Colab
# !pip install -q rheojax
# from google.colab import drive
# drive.mount('/content/drive')


## Setup and Imports
Use the PyVisco master-curve dataset to validate RheoJAX oscillatory fits and compare generalized vs fractional models.

In [2]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
%matplotlib inline

import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from rheojax.core.data import RheoData
from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fractional_maxwell_model import FractionalMaxwellModel
from rheojax.models.generalized_maxwell import GeneralizedMaxwell
from rheojax.pipeline.base import Pipeline
from rheojax.transforms.mastercurve import Mastercurve

jax, jnp = safe_import_jax()
verify_float64()
np.set_printoptions(precision=4, suppress=True)
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
warnings.filterwarnings('ignore', category=RuntimeWarning)

def r2_complex(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    ss_res = np.sum(np.abs(y_true - y_pred) ** 2)
    ss_tot = np.sum(np.abs(y_true - np.mean(y_true)) ** 2)
    return float(1 - ss_res / ss_tot)


INFO:2025-12-06 04:14:05,968:jax._src.xla_bridge:808: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache), 'libtpu.so' (no such file)


Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache), 'libtpu.so' (no such file)


Loading rheojax version 0.4.0


/Users/b80985/Projects/rheojax/.venv/lib/python3.13/site-packages/piblin_jax/__init__.py:28: UserWarning: GPU acceleration is only available on Linux with CUDA 12+. Using JAX in CPU mode.
  from . import backend, data, dataio, transform


## Load master-curve data

Files live in `examples/data/pyvisco/freq_master/` and include storage/loss moduli (MPa) plus tabulated shift factors.

In [3]:
DATA_DIR = Path.cwd().parent / 'data' / 'pyvisco' / 'freq_master'
master_df = pd.read_csv(DATA_DIR / 'freq_user_master.csv')
shift_df = pd.read_csv(DATA_DIR / 'freq_user_master__shift_factors.csv')

# Drop units row and coerce numeric
master_clean = master_df.iloc[1:].astype(float)

# Frequency in Hz → angular freq (rad/s)
freq_hz = master_clean['f'].to_numpy()
omega = 2 * np.pi * freq_hz
G_prime = master_clean['E_stor'].to_numpy()
G_double = master_clean['E_loss'].to_numpy()
G_star = G_prime + 1j * G_double

data = RheoData(
    x=omega,
    y=G_star,
    x_units='rad/s',
    y_units='MPa',
    domain='oscillation',
    metadata={'source': 'pyvisco_master', 'test_mode': 'oscillation'}
)
print(data)

# Display shift factors for reference
shift_df.head()


RheoData(x=array([6.2832e-12, 7.5541e-12, 1.2537e-11, 1.6275e-11, 2.7009e-11,
       3.5063e-11, 5.2261e-11, 5.8190e-11, 7.5541e-11, 1.1259e-10,
       1.2537e-10, 1.6275e-10, 2.4258e-10, 2.7009e-10, 3.5063e-10,
       4.1513e-10, 5.2261e-10, 5.8190e-10, 7.5541e-10, 8.9436e-10,
       1.1259e-09, 1.2537e-09, 1.6275e-09, 1.9268e-09, 2.1786e-09,
       2.4258e-09, 3.5063e-09, 4.1513e-09, 4.6937e-09, 5.2261e-09,
       7.5541e-09, 8.9436e-09, 1.0112e-08, 1.1259e-08, 1.1434e-08,
       1.9268e-08, 2.1786e-08, 2.4258e-08, 2.4633e-08, 4.1513e-08,
       4.6937e-08, 5.2261e-08, 5.3070e-08, 6.0004e-08, 8.9436e-08,
       1.0112e-07, 1.1434e-07, 1.2927e-07, 1.9268e-07, 2.1786e-07,
       2.4633e-07, 2.7427e-07, 2.7851e-07, 4.1513e-07, 4.6937e-07,
       5.3070e-07, 5.9090e-07, 6.0004e-07, 1.0112e-06, 1.1434e-06,
       1.2731e-06, 1.2927e-06, 1.4394e-06, 2.1786e-06, 2.4633e-06,
       2.7427e-06, 2.7851e-06, 3.1011e-06, 5.3070e-06, 5.9090e-06,
       6.0004e-06, 6.6811e-06, 8.6732e-06, 1.1434e-

,T,log_aT
0,C,-
1,100,-11.7
2,92,-10.92
3,85,-10.08
4,77,-9.18


## Visualize raw master curve

In [4]:
fig, ax = plt.subplots(figsize=(9, 6))
ax.loglog(freq_hz, G_prime, 'o', label="E' data")
ax.loglog(freq_hz, G_double, 's', label='E" data')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Modulus (MPa)')
ax.set_title('PyVisco master curve (storage/loss)')
ax.grid(True, which='both', ls='--', alpha=0.4)
ax.legend();


## Fit RheoJAX models

We compare a 6-mode generalized Maxwell model (Prony-style) against the fractional Maxwell model. Both use log-space residuals to respect the wide frequency range.

In [5]:
# Generalized Maxwell accepts complex modulus directly
gm = GeneralizedMaxwell(n_modes=6, modulus_type='tensile')
gm.fit(omega, G_star, test_mode='oscillation', use_log_residuals=True, use_multi_start=True)
gm_pred_components = gm.predict(omega)
gm_pred = gm_pred_components[:, 0] + 1j * gm_pred_components[:, 1]
gm_r2 = r2_complex(G_star, gm_pred)

# Fractional Maxwell (fixed-parameter variant) with graceful fallback
fm_pred = np.full_like(G_star, np.nan)
fm_r2 = np.nan
try:
    fm = FractionalMaxwellModel()
    fm.fit(omega, G_star, test_mode='oscillation', use_log_residuals=True)
    fm_pred = fm.predict(omega, test_mode='oscillation')
    fm_r2 = r2_complex(G_star, fm_pred)
except Exception as exc:
    print(f"Fractional Maxwell fit failed: {exc}")

print(f"Generalized Maxwell R^2: {gm_r2:.4f}")
print(f"Fractional Maxwell R^2:   {fm_r2 if np.isfinite(fm_r2) else float('nan'):.4f}")


Starting least squares optimization | {'method': 'trf', 'n_params': 13, 'loss': 'linear', 'ftol': 1e-06, 'xtol': 1e-06, 'gtol': 1e-06}


Timer: optimization took 1.162198s


Convergence: reason=`xtol` termination condition is satisfied. | iterations=11 | final_cost=8.202934e+07 | time=1.162s | final_gradient_norm=764963094.0502862


Starting least squares optimization | {'method': 'trf', 'n_params': 13, 'loss': 'linear', 'ftol': 1e-06, 'xtol': 1e-06, 'gtol': 1e-06}


Timer: optimization took 0.285773s


...

Auto-enabling multi-start optimization for very wide range (26.0 decades, 5 starts)


Starting least squares optimization | {'method': 'trf', 'n_params': 4, 'loss': 'linear', 'ftol': 1e-06, 'xtol': 1e-06, 'gtol': 1e-06}


Timer: optimization took 0.545556s


Convergence: reason=`xtol` termination condition is satisfied. | iterations=1 | final_cost=4.726795e+20 | time=0.546s | final_gradient_norm=1.569008955428314e+30


Fractional Maxwell fit failed: Optimization failed: residual norm remains extremely large. Try providing better initial values, looser bounds, or scaling the data.
Generalized Maxwell R^2: 0.9190
Fractional Maxwell R^2:   nan


## Overlay fits vs data

In [6]:
def plot_fit(ax, freq, data_complex, pred_complex, label):
    ax.loglog(freq, np.real(data_complex), 'o', alpha=0.3, label="E' data")
    ax.loglog(freq, np.imag(data_complex), 's', alpha=0.3, label='E" data')
    ax.loglog(freq, np.real(pred_complex), '-', label=f"E' {label}")
    ax.loglog(freq, np.imag(pred_complex), '--', label=f'E" {label}')
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('Modulus (MPa)')
    ax.grid(True, which='both', ls='--', alpha=0.4)

fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)
plot_fit(axes[0], freq_hz, G_star, gm_pred, 'Generalized Maxwell')
axes[0].set_title(f'Generalized Maxwell (R^2={gm_r2:.3f})')

if np.isfinite(fm_r2):
    plot_fit(axes[1], freq_hz, G_star, fm_pred, 'Fractional Maxwell')
    axes[1].set_title(f'Fractional Maxwell (R^2={fm_r2:.3f})')
else:
    axes[1].axis('off')
    axes[1].text(0.5, 0.5, 'Fractional fit failed', ha='center', va='center')

for ax in axes:
    ax.legend(loc='best')
plt.show()


/var/folders/rq/lgpr8fvj349cr_wvkzv99md80000gp/T/ipykernel_14144/2345376169.py:22: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(loc='best')
/var/folders/rq/lgpr8fvj349cr_wvkzv99md80000gp/T/ipykernel_14144/2345376169.py:23: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Residual diagnostics

In [7]:
def residual_mpe(y_true, y_pred):
    res = y_true - y_pred
    mpe = np.mean(np.abs(res) / np.maximum(np.abs(y_true), 1e-12)) * 100
    return float(mpe)

gm_mpe = residual_mpe(G_star, gm_pred)
fm_mpe = residual_mpe(G_star, fm_pred) if np.isfinite(fm_r2) else np.nan
print({'gm_mpe_%': gm_mpe, 'fm_mpe_%': fm_mpe})


{'gm_mpe_%': 57.71232110218008, 'fm_mpe_%': nan}
